In [1]:
import dash
from dash import html, dcc, Input, Output, State, ctx, ALL, MATCH
import plotly.express as px
import pandas as pd
from datetime import datetime

# Sample data
stockData = pd.DataFrame({
    'Name': ['A', 'B', 'C', 'D', 'E'],
    'Volume': [1000, 800, 600, 400, 200],
    '%Change_Bin': ['0%', '1%', '2%', '-1%', 'NaN'],
    'Open': [10, 20, 30, 40, 50],
    'PrevClose': [9, 19, 29, 39, 49],
    'ReferencePrice': [9.5, 19.5, 29.5, 39.5, 49.5],
    '%Change': [11, 5, -3, 0, None],
    'Change': [1, 1, -1, 0, 0],
    'Code': ['001', '002', '003', '004', '005']
})

# Color map helper

def get_discrete_color_map2(style='taiwan'):
    labels = ['≤-9%'] + [f"{i}%" for i in range(-9, 10)] + ['≥9%'] + ['NaN']
    if style == 'taiwan':
        colors = [
            "#003300", "#004d00", "#006600", "#008000", "#009900", "#00b300",
            "#00cc00", "#00e600", "#1aff1a", "#80ff80", "#e0e0e0",
            "#ffb3b3", "#ff9999", "#ff6666", "#ff4d4d", "#ff3333",
            "#ff1a1a", "#ff0000", "#e60000", "#cc0000", "#990000", "#808080"
        ]
    else:
        colors = [
            "#990000", "#cc0000", "#e60000", "#ff0000", "#ff1a1a", "#ff3333",
            "#ff4d4d", "#ff6666", "#ff9999", "#ffb3b3", "#e0e0e0",
            "#80ff80", "#1aff1a", "#00e600", "#00cc00", "#00b300",
            "#009900", "#008000", "#006600", "#004d00", "#003300", "#808080"
        ]
    return dict(zip(labels, colors))


def generate_color_legend(style='taiwan'):
    color_map = get_discrete_color_map2(style)
    items = []
    for label, color in color_map.items():
        safe = label.replace('%', 'pct').replace('=', 'eq').replace('>', 'gt').replace('<', 'lt')
        items.append(html.Div([
            html.Div(style={
                'display': 'inline-block',
                'width': '20px',
                'height': '20px',
                'backgroundColor': color,
                'marginRight': '10px',
                'border': '1px solid #ccc'
            }),
            html.Span(label),
            html.Button('', id={'type': 'legend-button', 'index': safe}, n_clicks=0,
                style={
                    'margin': '10px',
                    'padding': '10px 20px',
                    'fontSize': '16px',
                    'cursor': 'pointer',
                    'backgroundColor': color,
                    'color': 'black'
                }
            )
        ], style={'marginBottom': '5px'}))
    return html.Div(items)

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id='style-selector',
        options=[
            {'label': '台式配色（漲紅跌綠）', 'value': 'taiwan'},
            {'label': '國際配色（漲綠跌紅）', 'value': 'global'}
        ],
        value='taiwan'
    ),
    dcc.Graph(id='treemap-chart'),
    html.Div(id='legend', children=generate_color_legend())
])

@app.callback(
    Output({'type': 'legend-button', 'index': MATCH}, 'style'),
    Input({'type': 'legend-button', 'index': MATCH}, 'n_clicks'),
    State({'type': 'legend-button', 'index': MATCH}, 'style'),
    prevent_initial_call=True
)
def toggle_button(n, style):
    if style.get('backgroundColor') == 'white':
        return {**style, 'backgroundColor': style.get('_orig', '#ccc')}
    return {**style, '_orig': style.get('backgroundColor'), 'backgroundColor': 'white'}

@app.callback(
    Output('treemap-chart', 'figure'),
    Input('style-selector', 'value'),
    Input({'type': 'legend-button', 'index': ALL}, 'n_clicks'),
    State({'type': 'legend-button', 'index': ALL}, 'style')
)
def update_chart(style, _, all_styles):
    exclude = []
    for i, st in enumerate(all_styles):
        if st.get('backgroundColor') == 'white':
            exclude.append(ctx.inputs_list[1][i]['id']['index'].replace('pct','%').replace('eq','=').replace('gt','>').replace('lt','<'))
    df = stockData[~stockData['%Change_Bin'].isin(exclude)]
    fig = px.treemap(df, path=['Name'], values='Volume', color='%Change_Bin',
                     color_discrete_map=get_discrete_color_map2(style))
    fig.update_layout(margin=dict(t=5, l=5, r=5, b=5))
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

In [1]:
import dash
from dash import html, dcc, dash_table, Input, Output
import pandas as pd
import plotly.express as px

# Load stock data from CSV
stockData = pd.read_csv('stock_data.csv')

# Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H3('Select multiple stocks to display'),
    dcc.Dropdown(
        id='stock-selector',
        options=[{'label': name, 'value': name} for name in sorted(stockData['Name'].unique())],
        multi=True,
        placeholder='Choose stocks (leave empty to show all)'
    ),
    dcc.Graph(id='stock-chart'),
    dash_table.DataTable(id='stock-table', columns=[{'name': c, 'id': c} for c in stockData.columns])
])

@app.callback(
    Output('stock-chart', 'figure'),
    Output('stock-table', 'data'),
    Input('stock-selector', 'value')
)
def update_output(selected):
    if not selected:
        df = stockData
    else:
        df = stockData[stockData['Name'].isin(selected)]

    fig = px.bar(df, x='Name', y='TradeVolume', title='Trade Volume by Stock')
    return fig, df.to_dict('records')

if __name__ == '__main__':
    app.run_server(debug=True)